In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [3]:
# 텐서플로우에서 제공하는 MNIST데이터를 사용하겠다
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# 이미지 데이터의 픽섹 범위는 0~255이므로 255로 나누어 0~1사이 범위로 변경 255.0은 샘플 데이터를 정수에서 부동소수점 숫자로 바꾸어준다.
x_train, x_test = x_train / 255.0, x_test / 255.0

#  차원 추가
print(x_train.shape)
x_train = x_train[..., tf.newaxis].astype('float32')
# x_train = x_train[:500]
# y_train = y_train[:500]
x_test = x_test[..., tf.newaxis].astype('float32')
print(x_train.shape)


(60000, 28, 28)
(60000, 28, 28, 1)


In [ ]:
# x_train[:500].shape

In [ ]:
x_train.shape

In [4]:
# 앞에서 shuffle에 들어가는 숫자 만큼의 데이터를 뽑은 후 랜덤 배치를 구성(배치에 사용된 만큼 뒤에 데이터를 추가하여 새로운 배치를 뽑기  반복)
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(buffer_size=1024).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices(
    (x_test, y_test)).shuffle(buffer_size=1024).batch(32)


In [ ]:
# 위의 동작을 테스트 해보기 위한 코드
test = tf.data.Dataset.from_tensor_slices(
    [1,2,3,4,5,6,7,8,9]).shuffle(3).batch(5)
for k,k2 in train_ds:
    print(k)
    print(k2)
    break

In [5]:
class mymodel(Model):
    def __init__(self):
        super(mymodel, self).__init__() 
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)
    
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
    
model = mymodel()

In [6]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [7]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_acuuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_acuuracy')

In [8]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        pred = model(images, training=True)
        loss = loss_func(labels, pred)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, pred)


In [9]:
@tf.function
def test_step(images, labels):
    pred = model(images, training=False)
    t_loss = loss_func(labels, pred)
    
    test_loss(t_loss)
    test_accuracy(labels, pred)

In [ ]:
epochs = 5
for epoch in range(epochs):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    for images, labels in train_ds:
        train_step(images, labels)
#     print('?')
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)
#         print('?')
    print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )